In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt


In [165]:
train_df=pd.read_csv("df_hack_final.csv")
test_df=pd.read_csv("test.csv")

In [3]:
train_df.head()

,MEAS_DT,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
0,2024-01-01 00:00:00,2.6097,1.5313,1096.5,1240.597656,692.090942,1.342155,711.999023,1548.71875,0.6232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-01 00:15:00,2.5548,1.4842,1123.0,1205.422363,693.616394,1.339809,710.697815,1556.56250,0.6292,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,NaN
2,2024-01-01 00:30:00,2.5109,1.4355,840.0,1188.762573,698.350586,1.339792,707.198547,1548.09375,0.5941,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.970170
3,2024-01-01 00:45:00,2.4765,1.3852,824.0,1151.888672,714.678101,1.342392,707.865540,1538.87500,0.6682,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.968639
4,2024-01-01 01:00:00,2.3585,1.3368,0.0,1104.101318,730.190674,1.337608,700.935059,1545.18750,0.6489,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.974205


In [4]:
test_df.head()

,MEAS_DT,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,...,Ni_5.2C_min,Ni_5.2C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max,Ni_6.2T_min,Ni_6.2T_max,Ni_6.2C_min,Ni_6.2C_max
0,2024-01-19 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-19 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-19 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-19 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-19 13:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Поскольку мы знаем, что большинство фичей распределены нормально мы можем добавить и заполнить в тесте колонки признаков из train_df

# Заполнение теста

In [166]:
test_columns = [ 'MEAS_DT',
    'Ni_1.1C_min', 'Ni_1.1C_max', 'Cu_1.1C_min', 'Cu_1.1C_max',
    'Ni_1.2C_min', 'Ni_1.2C_max', 'Cu_1.2C_min', 'Cu_1.2C_max',
    'Cu_2.1T_min', 'Cu_2.1T_max', 'Cu_2.2T_min', 'Cu_2.2T_max',
    'Cu_3.1T_min', 'Cu_3.1T_max', 'Cu_3.2T_min', 'Cu_3.2T_max',
    'Ni_4.1T_min', 'Ni_4.1T_max', 'Ni_4.1C_min', 'Ni_4.1C_max',
    'Ni_4.2T_min', 'Ni_4.2T_max', 'Ni_4.2C_min', 'Ni_4.2C_max',
    'Ni_5.1T_min', 'Ni_5.1T_max', 'Ni_5.1C_min', 'Ni_5.1C_max',
    'Ni_5.2T_min', 'Ni_5.2T_max', 'Ni_5.2C_min', 'Ni_5.2C_max',
    'Ni_6.1T_min', 'Ni_6.1T_max', 'Ni_6.1C_min', 'Ni_6.1C_max',
    'Ni_6.2T_min', 'Ni_6.2T_max', 'Ni_6.2C_min', 'Ni_6.2C_max'
]


In [167]:
feature_columns = [col for col in train_df.columns if col not in test_columns]
median_features = train_df[feature_columns].median()

np.random.seed(42)
for column in feature_columns:
    if column not in test_df.columns:
        noise = np.random.normal(0, 0.05 * median_features[column], size=test_df.shape[0])
        test_df[column] = median_features[column] + noise



In [168]:
# Выделим колонки для которых мы будем делать предикты
target_columns = [ 
    'Ni_1.1C_min', 'Ni_1.1C_max', 'Cu_1.1C_min', 'Cu_1.1C_max',
    'Ni_1.2C_min', 'Ni_1.2C_max', 'Cu_1.2C_min', 'Cu_1.2C_max',
    'Cu_2.1T_min', 'Cu_2.1T_max', 'Cu_2.2T_min', 'Cu_2.2T_max',
    'Cu_3.1T_min', 'Cu_3.1T_max', 'Cu_3.2T_min', 'Cu_3.2T_max',
    'Ni_4.1T_min', 'Ni_4.1T_max', 'Ni_4.1C_min', 'Ni_4.1C_max',
    'Ni_4.2T_min', 'Ni_4.2T_max', 'Ni_4.2C_min', 'Ni_4.2C_max',
    'Ni_5.1T_min', 'Ni_5.1T_max', 'Ni_5.1C_min', 'Ni_5.1C_max',
    'Ni_5.2T_min', 'Ni_5.2T_max', 'Ni_5.2C_min', 'Ni_5.2C_max',
    'Ni_6.1T_min', 'Ni_6.1T_max', 'Ni_6.1C_min', 'Ni_6.1C_max',
    'Ni_6.2T_min', 'Ni_6.2T_max', 'Ni_6.2C_min', 'Ni_6.2C_max'
]


In [1]:
# Поскольку у нас есть ограничения на изменения для каждого порога концентрации, введём их и будем учитывать
min_deltas = {
    'Ni_1.1C_min': 0.1,
    'Ni_1.1C_max': 0.1,
    'Ni_1.2C_min': 0.1,
    'Ni_1.2C_max': 0.1,
    'Cu_1.1C_min': 0.1,
    'Cu_1.1C_max': 0.1,
    'Cu_1.2C_min': 0.1,
    'Cu_1.2C_max': 0.1,
    'Cu_2.1T_min': 0.01,
    'Cu_2.1T_max': 0.01,
    'Cu_2.2T_min': 0.01,
    'Cu_2.2T_max': 0.01,
    'Cu_3.1T_min': 0.05,
    'Cu_3.1T_max': 0.05,
    'Cu_3.2T_min': 0.05,
    'Cu_3.2T_max': 0.05,
    'Ni_4.1T_min': 0.01,
    'Ni_4.1T_max': 0.01,
    'Ni_4.2C_min': 0.05,
    'Ni_4.2C_max': 0.05,
    'Ni_5.1T_min': 0.01,
    'Ni_5.1T_max': 0.01,
    'Ni_5.2T_min': 0.01,
    'Ni_5.2T_max': 0.01,
    'Ni_5.1C_min': 0.05,
    'Ni_5.1C_max': 0.05,
    'Ni_5.2C_min': 0.05,
    'Ni_5.2C_max': 0.05,
    'Ni_6.1T_min': 0.01,
    'Ni_6.1T_max': 0.01,
    'Ni_6.2T_min': 0.01,
    'Ni_6.2T_max': 0.01,
    'Ni_6.1C_min': 0.05,
    'Ni_6.1C_max': 0.05,
    'Ni_6.2C_min': 0.05,
    'Ni_6.2C_max': 0.05
}



In [182]:
def smooth_predictions(predictions, timestamps, column_name, min_deltas, min_time_gap=64):
    """
    Ограничивает изменение значений предсказаний между соседними элементами
    с учётом минимальных допустимых приращений для конкретных признаков
    и минимального временного интервала между изменениями (в 15-минутных интервалах).
    
    Args:
        predictions (list or numpy array): Массив предсказанных значений.
        timestamps (list or pandas.Series): Временные метки предсказаний.
        column_name (str): Имя признака, для которого сглаживаются значения.
        min_deltas (dict): Словарь минимальных допустимых приращений для признаков.
        min_time_gap (int): Минимальное количество 15-минутных интервалов между изменениями.
    
    Returns:
        numpy array: Массив с "сглаженными" значениями.
    """
    # Минимальное приращение для текущего признака
    min_delta = min_deltas.get(column_name, 0.01)  # По умолчанию 0.01
    smoothed = predictions.copy()
    last_update_time = pd.to_datetime(timestamps[0])  # Инициализация первого времени
    
    for i in range(1, len(smoothed)):
        # Преобразование временных меток в datetime
        current_time = pd.to_datetime(timestamps[i])
        time_gap = (current_time - last_update_time).total_seconds() / (15 * 60)  # Интервалы в 15 минутах
        delta = smoothed[i] - smoothed[i - 1]

        # Проверяем, можно ли менять значение
        if time_gap >= min_time_gap:
            if delta > 0:
                allowed_delta = min_delta * round(delta / min_delta)
                smoothed[i] = smoothed[i - 1] + min(allowed_delta, delta)
            elif delta < 0:
                allowed_delta = min_delta * round(delta / min_delta)
                smoothed[i] = smoothed[i - 1] + max(allowed_delta, delta)
            
            # Обновляем время последнего изменения
            last_update_time = current_time
        else:
            # В течение минимального временного интервала оставляем предыдущее значение
            smoothed[i] = smoothed[i - 1]
    
    return smoothed

# Модель

В качестве модели мы будем использовать CatBoostRegressor и будем предиктить каждое из значений пороговых концентраций отдельной моделью

In [183]:
models = {}

for target in target_columns:
    print(f"Training model for {target}...")
    
    # Убираем строки с пропусками в текущем таргете
    train_filtered = train_df.dropna(subset=[target])
    
    # Разделяем на признаки и целевую переменную
    X_train = train_filtered[feature_columns]
    y_train = train_filtered[target]
    
    # Обучение модели
    model = CatBoostRegressor(
        loss_function="RMSE",
        iterations=500,
        learning_rate=0.05,
        depth=6,
        random_seed=42,
        verbose=0
    )
    model.fit(X_train, y_train)
    models[target] = model  # Сохраняем модель

    # Предсказание для теста
    X_test = test_df[feature_columns]
    predictions = model.predict(X_test)

    # Получаем временные метки из тестового набора
    timestamps = test_df['MEAS_DT']
    
    # Применение сглаживания
    test_df[target] = smooth_predictions(
        predictions=predictions,
        timestamps=timestamps,
        column_name=target,
        min_deltas=min_deltas,
        min_time_gap=16  # Например, 8 15-минутных интервалов (2 часа)
    )
    if len(models) == 1:
        feature_importances = model.get_feature_importance(prettified=True)
        top_10_features = feature_importances.sort_values(by='Importances', ascending=False).head(10)
        print(f"Top 10 features for {target}:")
        print(top_10_features)



Training model for Ni_1.1C_min...
Top 10 features for Ni_1.1C_min:
    Feature Id  Importances
0  Cu_2.2C_max    51.612764
1  Cu_3.1C_max    14.218777
2  Cu_3.1C_min     6.934386
3  Cu_3.2C_max     6.912767
4  Cu_3.2C_min     5.441495
5  Ni_1.1T_max     3.868332
6        Ni_2F     2.649480
7  Ni_1.2T_max     1.542417
8     Cu_resth     1.386908
9      Cu_1.2C     1.059169
Training model for Ni_1.1C_max...
Training model for Cu_1.1C_min...
Training model for Cu_1.1C_max...
Training model for Ni_1.2C_min...
Training model for Ni_1.2C_max...
Training model for Cu_1.2C_min...
Training model for Cu_1.2C_max...
Training model for Cu_2.1T_min...
Training model for Cu_2.1T_max...
Training model for Cu_2.2T_min...
Training model for Cu_2.2T_max...
Training model for Cu_3.1T_min...
Training model for Cu_3.1T_max...
Training model for Cu_3.2T_min...
Training model for Cu_3.2T_max...
Training model for Ni_4.1T_min...
Training model for Ni_4.1T_max...
Training model for Ni_4.1C_min...
Training mode

Из-за ограничений мы должны будем округлить полученные значения в зависимости от концентраций

In [184]:
# Списки колонок с разным правилом округления

round_1_columns = [
    'Ni_1.1C_min', 'Ni_1.1C_max',
    'Ni_1.2C_min', 'Ni_1.2C_max',
    'Cu_1.1C_min', 'Cu_1.1C_max',
    'Cu_1.2C_min', 'Cu_1.2C_max'
]

# Округляем
for target in target_columns:
    if target in round_1_columns:
        test_df[target] = test_df[target].round(1) 
    else:
        test_df[target] = test_df[target].round(2) 

In [185]:
test_df.head()

,MEAS_DT,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,...,Cu_3.1C_max,Cu_3.1C_min,Ni_3.1C_max,Ni_3.1C_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Ni_rec
0,2024-01-19 12:15:00,2.7,3.2,4.4,4.9,2.7,3.4,4.6,5.1,0.26,...,16.764210,13.118578,3.876161,3.721875,0.0,16.376349,12.484377,3.808726,3.594361,0.995530
1,2024-01-19 12:30:00,2.7,3.2,4.4,4.9,2.7,3.4,4.6,5.1,0.26,...,16.889844,13.257914,3.678761,3.607911,0.0,15.157184,13.430080,3.556178,3.425023,0.954710
2,2024-01-19 12:45:00,2.7,3.2,4.4,4.9,2.7,3.4,4.6,5.1,0.26,...,14.777844,13.284360,4.115661,3.546197,0.0,16.209709,13.998074,3.601005,3.492632,0.929124
3,2024-01-19 13:00:00,2.7,3.2,4.4,4.9,2.7,3.4,4.6,5.1,0.26,...,16.342267,13.257680,3.583503,3.852921,0.0,16.085169,12.803788,3.912534,3.400722,0.882552
4,2024-01-19 13:15:00,2.7,3.2,4.4,4.9,2.7,3.4,4.6,5.1,0.26,...,16.160332,12.419480,3.719183,3.940161,0.0,15.048785,12.370216,3.422530,3.495250,0.976547


# Дополнительные фичи

### Время суток
Возможно при смене технолога происходят какие-то значимые изменения и мы сможем их уловить 

In [127]:
test_df['MEAS_DT'] = pd.to_datetime(test_df['MEAS_DT'], errors='coerce')
train_df['MEAS_DT'] = pd.to_datetime(train_df['MEAS_DT'], errors='coerce')
test_df['hour'] = test_df['MEAS_DT'].dt.hour
train_df['hour'] = train_df['MEAS_DT'].dt.hour


### Нормированная длина диапазона относительно концентрации
Эта фича может быть полезна для того, чтобы понимать нужно нам уменьшать диапазон или нет

In [128]:
concentration_triples = [
    ('Ni_1.1C_min', 'Ni_1.1C_max', 'Ni_1.1C'),
    ('Ni_1.2C_min', 'Ni_1.2C_max', 'Ni_1.2C'),
    ('Cu_1.1C_min', 'Cu_1.1C_max', 'Cu_1.1C'),
    ('Cu_1.2C_min', 'Cu_1.2C_max', 'Cu_1.2C'),
    ('Cu_2.1T_min', 'Cu_2.1T_max', 'Cu_2.1T'),
    ('Cu_2.2T_min', 'Cu_2.2T_max', 'Cu_2.2T'),
    ('Cu_3.1T_min', 'Cu_3.1T_max', 'Cu_3.1T'),
    ('Cu_3.2T_min', 'Cu_3.2T_max', 'Cu_3.2T'),
    ('Ni_4.1T_min', 'Ni_4.1T_max', 'Ni_4.1T'),
    ('Ni_4.1C_min', 'Ni_4.1C_max', 'Ni_4.1C'),
    ('Ni_4.2T_min', 'Ni_4.2T_max', 'Ni_4.2T'),
    ('Ni_4.2C_min', 'Ni_4.2C_max', 'Ni_4.2C'),
    ('Ni_5.1T_min', 'Ni_5.1T_max', 'Ni_5.1T'),
    ('Ni_5.1C_min', 'Ni_5.1C_max', 'Ni_5.1C'),
    ('Ni_5.2T_min', 'Ni_5.2T_max', 'Ni_5.2T'),
    ('Ni_5.2C_min', 'Ni_5.2C_max', 'Ni_5.2C'),
    ('Ni_6.1T_min', 'Ni_6.1T_max', 'Ni_6.1T'),
    ('Ni_6.1C_min', 'Ni_6.1C_max', 'Ni_6.1C'),
    ('Ni_6.2T_min', 'Ni_6.2T_max', 'Ni_6.2T'),
    ('Ni_6.2C_min', 'Ni_6.2C_max', 'Ni_6.2C')
]
# Нормированная длина диапазона относительно концентрации
for min_col, max_col, real_change_col in concentration_triples:
    feature_name = f"normalized_range_length_{real_change_col}"
    train_df[feature_name] = (
        (train_df[max_col] - train_df[min_col]) /
        (train_df[real_change_col] + 1e-6)  # Чтобы избежать деления на 0
    )
    test_df[feature_name] = (
        (test_df[max_col] - test_df[min_col]) /
        (test_df[real_change_col] + 1e-6)  # Чтобы избежать деления на 0
    )


C:\Users\User\AppData\Local\Temp\ipykernel_16672\4106684999.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[feature_name] = (
C:\Users\User\AppData\Local\Temp\ipykernel_16672\4106684999.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[feature_name] = (
C:\Users\User\AppData\Local\Temp\ipykernel_16672\4106684999.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

### Изменение концентрации за 2 часа
Это может быть полезно для понимания насколько нестабильно оценивалась концентрация металла в пульпе в течении времени, пока пороги менять было нельзя

In [129]:
def calculate_concentration_changes(df, concentration_triples, time_column, min_interval_minutes=15):
    """
    Функция для вычисления изменения концентрации за 2 часа для каждого столбца из concentration_triples.
    
    Args:
        df (DataFrame): Датафрейм с данными.
        concentration_triples (list): Список кортежей, каждый содержит (min_column, max_column, real_concentration_column).
        time_column (str): Столбец с временными метками.
        min_interval_minutes (int): Минимальный интервал между измерениями (в минутах).
    
    Returns:
        DataFrame: Обновлённый DataFrame с добавленными изменениями концентрации.
    """
    df = df.sort_values(by=time_column)
    
    df[time_column] = pd.to_datetime(df[time_column])
    
    # Добавляем колонку с разницей во времени между соседними измерениями
    df['time_diff'] = df[time_column].diff().dt.total_seconds() / 60  # в минутах

    # Создаём для каждого real_col новый столбец изменения концентрации за 2 часа
    for min_col, max_col, real_col in concentration_triples:
        # Сдвигаем данные по времени на 2 часа назад для вычисления изменений
        df[f"{real_col}_change_2h"] = df[real_col] - df[real_col].shift(periods=8)  # 8 интервалов по 15 минут = 2 часа
        
        # Применяем условие для записи изменений только для тех, где разница во времени >= 120 минут (2 часа)
        df[f"{real_col}_change_2h"] = df.apply(
            lambda row: row[f"{real_col}_change_2h"] if row['time_diff'] >= 120 else None, axis=1
        )
    
    df = df.drop(columns=['time_diff'])
    
    return df

train_df = calculate_concentration_changes(train_df, concentration_triples, 'MEAS_DT')
test_df = calculate_concentration_changes(test_df, concentration_triples, 'MEAS_DT')


### Диапазон между порогами концентрации, нормированный на изменение концентрации за 2 часа

In [130]:
def calculate_normalized_range_change(df, concentration_triples):
    """
    Вычисление изменения диапазона между порогами концентрации, нормированное на изменение концентрации за 2 часа.

    Args:
        df (DataFrame): Датафрейм с данными.
        concentration_triples (list): Список кортежей, каждый содержит (min_column, max_column, real_concentration_column).

    Returns:
        DataFrame: Обновлённый DataFrame с добавленными признаками нормированного изменения диапазона.
    """
    for min_col, max_col, real_change_col in concentration_triples:
        feature_name = f"2h_normalized_range_length_{real_change_col}"
        
        df[feature_name] = (
            (df[max_col] - df[min_col]) / 
            (df[f"{real_change_col}_change_2h"] + 1e-6)  # Добавляем малое значение для избегания деления на 0
        )
    
    return df

train_df = calculate_normalized_range_change(train_df, concentration_triples)
test_df = calculate_normalized_range_change(test_df, concentration_triples)


In [131]:
old_train=pd.read_csv("df_hack_final.csv")

In [132]:
new_feature_columns = [col for col in train_df.columns if col not in old_train.columns]


In [133]:
feature_columns=feature_columns+new_feature_columns

In [22]:
models = {}

for target in target_columns:
    print(f"Training model for {target}...")
    
    # Убираем строки с пропусками в текущем таргете
    train_filtered = train_df.dropna(subset=[target])
    
    # Разделяем на признаки и целевую переменную
    X_train = train_filtered[feature_columns]
    y_train = train_filtered[target]
    
    # Обучение модели
    model = CatBoostRegressor(
        loss_function="RMSE",
        iterations=500,
        learning_rate=0.05,
        depth=6,
        random_seed=42,
        verbose=0
    )
    model.fit(X_train, y_train)
    models[target] = model  # Сохраняем модель

    # Предсказание для теста
    X_test = test_df[feature_columns]
    predictions = model.predict(X_test)

    # Получаем временные метки из тестового набора
    timestamps = test_df['MEAS_DT']
    
    # Применение сглаживания
    test_df[target] = smooth_predictions(
        predictions=predictions,
        timestamps=timestamps,
        column_name=target,
        min_deltas=min_deltas,
        min_time_gap=8  # Например, 8 15-минутных интервалов (2 часа)
    )
    if len(models) == 1:
        feature_importances = model.get_feature_importance(prettified=True)
        top_10_features = feature_importances.sort_values(by='Importances', ascending=False).head(10)
        print(f"Top 10 features for {target}:")
        print(top_10_features)



Training model for Ni_1.1C_min...
Top 10 features for Ni_1.1C_min:
                        Feature Id  Importances
0                      Cu_2.2C_max    68.376988
1  normalized_range_length_Ni_5.2C     6.978453
2  normalized_range_length_Cu_1.1C     3.871429
3  normalized_range_length_Ni_1.1C     3.586946
4                      Ni_1.1T_max     2.487824
5                      Ni_1.2T_max     2.470530
6                      Cu_3.2C_max     2.422933
7  normalized_range_length_Ni_5.1C     1.987297
8                      Cu_3.1C_max     0.913814
9  normalized_range_length_Ni_6.2T     0.912701
Training model for Ni_1.1C_max...
Training model for Cu_1.1C_min...
Training model for Cu_1.1C_max...
Training model for Ni_1.2C_min...
Training model for Ni_1.2C_max...
Training model for Cu_1.2C_min...
Training model for Cu_1.2C_max...
Training model for Cu_2.1T_min...
Training model for Cu_2.1T_max...
Training model for Cu_2.2T_min...
Training model for Cu_2.2T_max...
Training model for Cu_3.1T_min.

In [25]:
round_1_columns = [
    'Ni_1.1C_min', 'Ni_1.1C_max',
    'Ni_1.2C_min', 'Ni_1.2C_max',
    'Cu_1.1C_min', 'Cu_1.1C_max',
    'Cu_1.2C_min', 'Cu_1.2C_max'
]

# Округляем
for target in target_columns:
    if target in round_1_columns:
        test_df[target] = test_df[target].round(1) 
    else:
        test_df[target] = test_df[target].round(2) 

# Выводы
- Поскольку у нас нет фичей, которые есть в трейне, но есть знание, что они исторически похожи, то мы заполнили ими тест
- В качестве таргета выбрали пороги концентраций 
- Использовали CatBoostRegressor для получения значений каждой из фичи
- Добавили новые фичи, такие как время суток, диапазон между порогами концентрации, нормированный на изменение концентрации за 2 часа, изменение концентрации за 2 часа, нормированная длина диапазона относительно концентрации
- Некоторые из добавленных фичей вошли в топ-10 по feature_importance